In [1]:
import re
import nltk

from typesql.utils import *

In [2]:
sql_data, table_data, val_sql_data, val_table_data, \
            test_sql_data, test_table_data, \
            TRAIN_DB, DEV_DB, TEST_DB = load_dataset(use_small=True) # False (full dataset)

Loading from original dataset
Loading data from data/train_tok.jsonl
Loading data from data/train_tok.tables.jsonl
Loading data from data/dev_tok.jsonl
Loading data from data/dev_tok.tables.jsonl
Loading data from data/test_tok.jsonl
Loading data from data/test_tok.tables.jsonl


In [4]:
sql_data[0:20]

[{'question_type_concol_list': [['te8r2ed'],
   ['te8r2ed'],
   ['te8r2ed'],
   ['column'],
   ['te8r2ed'],
   ['te8r2ed'],
   ['notes'],
   ['te8r2ed']],
  'kg_entities': [],
  'question_type_concol': ['te8r2ed',
   'te8r2ed',
   'te8r2ed',
   'column',
   'te8r2ed',
   'te8r2ed',
   'entity',
   'te8r2ed'],
  'question_type_org_kgcol': ['te8r2ed',
   'te8r2ed',
   'te8r2ed',
   'column',
   'column',
   'te8r2ed',
   'te8r2ed',
   'te8r2ed',
   'te8r2ed',
   'te8r2ed',
   'te8r2ed',
   'date',
   'date',
   'te8r2ed'],
  'question_tok_concol': [['what'],
   ['is'],
   ['the'],
   ['current', 'series'],
   ['where'],
   ['the'],
   ['new', 'series', 'began', 'in', 'june', '2011'],
   ['?']],
  'question': 'What is the current series where the new series began in June 2011?',
  'question_tok_kgcol': [['what'],
   ['is'],
   ['the'],
   ['current', 'series'],
   ['where'],
   ['the'],
   ['new'],
   ['series'],
   ['began'],
   ['in'],
   ['june', '2011'],
   ['?']],
  'question_type_kg

In [ ]:
def concatenate_sql_data(sql_data_train, sql_data_val):
    sql_data_train.extend(sql_data_val)
    return sql_data_train

In [9]:
def annotate_pos(question_tok_concol, pos_tags):
    pos_concol = list()
    tok_span = 0
    for tok_concol in question_tok_concol:
        if len(tok_concol) > 1:
            tok_span += len(tok_concol)
            pos_concol.append([pos for pos in pos_tags[tok_span-len(tok_concol):tok_span]])
        else:
            pos_concol.append(pos_tags[tok_span].split())
            tok_span += 1
    return pos_concol

In [10]:
def update_sql_data_pos(sql_data):
    """
    Update SQL data with POS tags for each question.
    """
    for i, question in enumerate(sql_data):
        tok_pos_tags = nltk.pos_tag(question['question_tok'])
        _, pos_tags = zip(*tok_pos_tags)
        assert len(question['question_tok']) == len(pos_tags)
        sql_data[i]['question_tok_pos'] = list(pos_tags)
        pos_tags_concol = annotate_pos(question['question_tok_concol'], list(pos_tags))
        assert len(question['question_tok_concol']) == len(pos_tags_concol)
        sql_data[i]['question_tok_concol_pos'] = pos_tags_concol
    return sql_data

In [11]:
sql_data = update_sql_data_pos(sql_data)